<a name="top"> <h1>02. Neuronal Networks</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>

<p>&nbsp;</p>

<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[1. Librerias utilizadas y funciones](#librerias) 

[2. Introducción ](#introduccion) 

   - Objetivo de negocio.

[3. Yelp Dataset ](#yelp) 

   - Información del dataset
   - Características del dataset


[4. Transformación del formato de ficheros](#transformacion) 


[5. Transformación de datos](#datos)

   - Business
       - Carga del fichero
       - Transformación de los datos
       - Exportación de ficheros procesados

<hr style="border:1px solid gray">

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para el preprocesamiento:

In [2]:
# Import libraries.

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import numpy as np
import string

# Import neural network libraries.

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# <a name="lectura"> 2. Lectura del dataframe <a>


In [4]:
#Import parquet file.

reviews = pd.read_parquet('reviews.parquet')

# Show the head of the dataframe.

reviews.head()

,reviewText,overall
0,amazon shopping amazoncom gift cards christmas...,1
1,got gift card friend best site much choose gre...,5
2,arent going save trees people complaining pape...,5
3,always get someone something amazon safety net...,5
4,take 50 dollars good money limitations turn am...,1


In [5]:
X = reviews['reviewText'].values
y = reviews['overall'].values

In [6]:
y = tf.keras.utils.to_categorical(y-1, num_classes=5)

Convertimos las putnuaciones a onehot encoding

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


In [8]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


In [9]:
X_train = pad_sequences(X_train, maxlen=300)
X_test = pad_sequences(X_test, maxlen=300)

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=300),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(5, activation='softmax')
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)


Epoch 1/10
920/920 [==============================] - 75s 69ms/step - loss: 0.4372 - accuracy: 0.8803 - val_loss: 0.3854 - val_accuracy: 0.8903
Epoch 2/10
920/920 [==============================] - 18s 20ms/step - loss: 0.3699 - accuracy: 0.8928 - val_loss: 0.3803 - val_accuracy: 0.8902
Epoch 3/10
920/920 [==============================] - 16s 18ms/step - loss: 0.3469 - accuracy: 0.8978 - val_loss: 0.3762 - val_accuracy: 0.8925
Epoch 4/10
920/920 [==============================] - 14s 15ms/step - loss: 0.3318 - accuracy: 0.9016 - val_loss: 0.3787 - val_accuracy: 0.8919
Epoch 5/10
920/920 [==============================] - 14s 15ms/step - loss: 0.3185 - accuracy: 0.9052 - val_loss: 0.3833 - val_accuracy: 0.8889
Epoch 6/10
920/920 [==============================] - 13s 14ms/step - loss: 0.3055 - accuracy: 0.9091 - val_loss: 0.3920 - val_accuracy: 0.8898
Epoch 7/10
920/920 [==============================] - 14s 15ms/step - loss: 0.2938 - accuracy: 0.9133 - val_loss: 0.3979 - val_accuracy:

In [12]:
# Guardar el modelo en un archivo
model.save("nn_reviews.h5")
